# Chapter 09: 에이전트 평가 (Evaluation)

이 노트북에서는 AI 에이전트의 성능을 평가하는 다양한 메트릭을 학습합니다.

## 주요 내용
- Phrase Recall (구문 재현율)
- Tool Metrics (도구 호출 정확도)
- BERTScore와 코사인 유사도

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch09_evaluation.ipynb)


In [ ]:
!pip install -q bert-score sentence-transformers


## 1. 평가 메트릭 정의


In [ ]:
from typing import Dict, List
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util

# Sentence Transformer 모델 로드
st_model = SentenceTransformer("all-MiniLM-L6-v2")

def phrase_recall(pred_reply: str, phrases: List[str]) -> float:
    """예측 응답에서 예상 구문들이 얼마나 포함되었는지 측정"""
    if not phrases:
        return 1.0
    found = sum(1 for p in phrases if p.lower() in pred_reply.lower())
    return found / len(phrases)

def tool_metrics(pred_tools: List[str], expected_calls: List[dict]) -> Dict[str, float]:
    """도구 호출의 정확도 측정"""
    expected_names = [c.get("tool") for c in expected_calls]
    if not expected_names:
        return {"tool_recall": 1.0, "tool_precision": 1.0}
    pred_set = set(pred_tools)
    exp_set = set(expected_names)
    tp = len(exp_set & pred_set)
    recall = tp / len(exp_set)
    precision = tp / len(pred_set) if pred_set else 0.0
    return {"tool_recall": recall, "tool_precision": precision}

def bert(pred, ref):
    """BERTScore를 사용한 텍스트 유사도"""
    P, R, F = bert_score([pred], [ref], lang="en", rescale_with_baseline=True)
    return F.mean().item()

def cosine(pred, ref):
    """임베딩 코사인 유사도"""
    emb = st_model.encode([pred, ref], convert_to_tensor=True, normalize_embeddings=True)
    return util.cos_sim(emb[0], emb[1]).item()

print("✅ 평가 메트릭 정의 완료")


## 2. 메트릭 테스트


In [ ]:
# 테스트 데이터
pred_response = "주문이 취소되었습니다. 환불이 3-5 영업일 내에 처리됩니다."
expected_phrases = ["취소", "환불", "영업일"]

pred_tools = ["cancel_order", "send_notification"]
expected_tool_calls = [
    {"tool": "cancel_order", "params": {"order_id": "123"}},
    {"tool": "issue_refund", "params": {"amount": 50.0}}
]

reference = "Your order has been cancelled. Refund will be processed within 3-5 business days."
prediction = "Order cancelled successfully. Expect refund in 3 to 5 working days."

# 메트릭 계산
print("=" * 50)
print("📊 평가 메트릭 결과")
print("=" * 50)

print(f"\n1. Phrase Recall: {phrase_recall(pred_response, expected_phrases):.2f}")
print(f"   (예상 구문 {len(expected_phrases)}개 중 포함된 수)")

tool_m = tool_metrics(pred_tools, expected_tool_calls)
print(f"\n2. Tool Recall: {tool_m['tool_recall']:.2f}")
print(f"   Tool Precision: {tool_m['tool_precision']:.2f}")

print(f"\n3. BERTScore: {bert(prediction, reference):.3f}")
print(f"   (의미적 유사도)")

print(f"\n4. Cosine Similarity: {cosine(prediction, reference):.3f}")
print(f"   (임베딩 유사도)")
